- 保监会 相关性模型 2 训练 

# 基本设置

In [2]:
##load packages, needed
# encoding=utf-8

import jieba
import sys
import re
import time
import string
from sklearn import feature_extraction
from sklearn.pipeline import Pipeline,FeatureUnion

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2,mutual_info_classif,f_classif 

import xgboost as xgb
from xgboost import XGBClassifier
from sklearn import metrics

from sklearn.base import BaseEstimator, TransformerMixin
from collections import defaultdict

import pickle

import joblib
%matplotlib inline
import numpy as np
import pandas as pd
import pre_cor
import os
from sklearn.model_selection import train_test_split
from sqlalchemy import create_engine
from pandas.io import sql
from collections import Counter

import warnings
warnings.filterwarnings('ignore')

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.897 seconds.
Prefix dict has been built succesfully.


In [42]:
class StatsFeatures(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        self.neg = set()
        f = open("corpus/neg_words.txt","r+", encoding='UTF-8')
        for content in f:
            self.neg.add(content)
        f.close()

    def fit(self, X, y=None):
        return self

    def getcnt(self,x):        
        return len(list(set(x)))

    def getnegcnt(self,x):
        negcnt = 0
        words = x.split()
        for w in words:
            if w in self.neg:
                negcnt = negcnt+1
        return negcnt
    
    def transform(self, X):
        data = []
        for x in X:
            if len(x) == 0:
                length  = 1
            else :
                length = len(x)
            data.append([len(x),self.getcnt(x),self.getcnt(x)/length,
                         self.getnegcnt(x),self.getnegcnt(x)/length])            
        return data

## 上一版模型读入

In [43]:
# 上一版模型
from sklearn.externals import joblib
pipeline_old = joblib.load( "model/circ_cor_0625.pkl.z") 

# 导入数据

In [4]:
folder = '20180703'

# 相关数据
corpus_cor = []
label_cor = []

filename = 'data/{0}/corpus_pre_cor_0703.txt'.format(folder)
fid = open(filename, "r+", encoding='UTF-8')
for f in fid:
    corpus_cor.append(f)
    label_cor.append(1)
fid.close()
print(len(corpus_cor))
print(len(label_cor))

6111
6111


In [5]:
# 不相关数据
corpus_uncor = []
label_uncor = []

filename = 'data/{0}/corpus_pre_uncor_0703.txt'.format(folder)
fid = open(filename, "r+", encoding='UTF-8')
for f in fid:
    corpus_uncor.append(f)
    label_uncor.append(0)
fid.close()
print(len(corpus_uncor))
print(len(label_uncor))

8949
8949


In [7]:
corpus_uncor[0]

'月 日 闽侯 万佛寺 祈福 永泰 赏 梅花 旗山 森林 温泉 度假村 品 永泰 葱饼 特价 一日游 特价 特价 特价 元 人 含 车费 原价 元 温泉 特惠 门票 活动 时间 年 月 日 星期天 集合时间 上午 集中 准时 出发 超时 不侯 集合地点 福州 工业 路 万象 城 麦当劳 东側 中央 第五 街 下面 上车 集中 指示 相片 活动 性质 自愿 参加 量力而行 保险 自购 风险 自负 户外 风险 参加 需谨慎 购买 保险 网址 https 活动 类别 感受 户外 生活 祈福 赏花 摄影 泡温泉 风景 指数 活动 强度 休闲游 亲子 游 线路 地点 福州 闽侯 永泰 福州 活动 费用 元 人 往返 车费 原 元 温泉 特惠 门票 米 以下 小孩 车费 元 人 景区 特价 活动 任何 证件 均 不再 优惠 上车 缴费 午餐 自理 自带 干粮 镇上 自行 午餐 按照 报名 先后顺序 安排 车上 座位 按车 均分 顺序 诚信 报名 报名者 如 活动 前 小时 内 退出 应 自觉 缴纳 本次 活动 车费 代 报名者 负责 补交 找 其他人 替补 请 大家 共同 遵守 报名 方式 请 参加者 发短信 报名 发短信 格式 日 旗山 温泉 网名 名字 qq 网名 参加 人数 接收 短信 手机 活动 行程 安排 早上 万象 城 发车 永泰 葛岭 赏花 和品 永泰 葱饼 上车 前往 游 旗山 万福 寺 镇上 自行 午餐 驱车 旗山 温泉 度假村 泡温泉 适时 返回 福州 线路 介绍 福州 城 四周 青山 耸翠 碧水 环绕 西郊 之旗 山群峰 旖旎 奇岩 幽壑 飞泉 流涧 景色 佳绝 鼓山 隔江 相峙 素有 左旗 右鼓 八闽 二绝 之誉 自古 道 天下 名山 僧 占 多 名山 古刹 相得益彰 也 唐代 以后 峰回 水曲 洞天福地 之中 旗山 曾 九庵 十八 寺 错落 其间 而 今天 要说 中国 占地 规模 最大 佛教 寺院 万佛寺 位于 旗山 之麓 万顷 石松 围 佛寺 一江 闽水绕 旗山 寺 前 对联 万佛 万玉 也 寺 中之佛 佛佛 皆 玉雕 而成 如今 海内外 佛教 信徒 朝拜 圣地 也 成为 东南 旅游 重要 景观 永泰 成片 梅林 中 一朵朵 白 梅花 密密麻麻 点缀 枝丫 间 远远 看去 犹如 刚下 一场 冬雪 整座 山 变成 白色 一株株 梅树 挺直 腰杆 一颗颗

# 训练模型

## 分割训练集、测试集

In [47]:
# 未加入系统中噪音
corpus = corpus_cor + corpus_uncor
label = label_cor + label_uncor
X_train, X_test, y_train, y_test = train_test_split(corpus, label, test_size=0.1, random_state=42)
print('训练集：',len(y_train))
print('训练集-各类数量：',Counter(y_train))
print('测试集：',len(y_test))
print('测试集-各类数量：',Counter(y_test))

训练集： 13554
训练集-各类数量： Counter({0: 8087, 1: 5467})
测试集： 1506
测试集-各类数量： Counter({0: 862, 1: 644})


## train

In [10]:
vectorizer=CountVectorizer()

In [13]:
vec = vectorizer.fit(corpus_uncor)

In [15]:
vec.vocabulary_

{'闽侯': 144669,
 '万佛寺': 17574,
 '祈福': 113452,
 '永泰': 97218,
 '梅花': 93864,
 '旗山': 85612,
 '森林': 94193,
 '温泉': 101961,
 '度假村': 70025,
 '葱饼': 127603,
 '特价': 105483,
 '一日游': 16636,
 '车费': 136361,
 '原价': 45419,
 '特惠': 105575,
 '门票': 144296,
 '活动': 99800,
 '时间': 86986,
 '星期天': 87475,
 '集合时间': 146827,
 '上午': 18606,
 '集中': 146785,
 '准时': 37598,
 '出发': 38288,
 '超时': 135117,
 '不侯': 19375,
 '集合地点': 146826,
 '福州': 113846,
 '工业': 67005,
 '万象': 17848,
 '麦当劳': 152054,
 '东側': 20915,
 '中央': 22337,
 '第五': 116224,
 '下面': 19272,
 '上车': 18923,
 '指示': 80253,
 '相片': 111448,
 '性质': 74263,
 '自愿': 124530,
 '参加': 45788,
 '量力而行': 141957,
 '保险': 31938,
 '自购': 124769,
 '风险': 149394,
 '自负': 124765,
 '户外': 76587,
 '需谨慎': 147406,
 '购买': 133614,
 '网址': 120383,
 'https': 5869,
 '类别': 117112,
 '感受': 75494,
 '生活': 108076,
 '赏花': 134213,
 '摄影': 82131,
 '泡温泉': 99067,
 '风景': 149314,
 '指数': 80219,
 '强度': 71754,
 '休闲游': 29281,
 '亲子': 26804,
 '线路': 118770,
 '地点': 54661,
 '费用': 133946,
 '往返': 72421,
 '以下': 28460,
 '小孩': 64946,
 '

In [14]:
help(vec)

Help on CountVectorizer in module sklearn.feature_extraction.text object:

class CountVectorizer(sklearn.base.BaseEstimator, VectorizerMixin)
 |  Convert a collection of text documents to a matrix of token counts
 |  
 |  This implementation produces a sparse representation of the counts using
 |  scipy.sparse.csr_matrix.
 |  
 |  If you do not provide an a-priori dictionary and you do not use an analyzer
 |  that does some kind of feature selection then the number of features will
 |  be equal to the vocabulary size found by analyzing the data.
 |  
 |  Read more in the :ref:`User Guide <text_feature_extraction>`.
 |  
 |  Parameters
 |  ----------
 |  input : string {'filename', 'file', 'content'}
 |      If 'filename', the sequence passed as an argument to fit is
 |      expected to be a list of filenames that need reading to fetch
 |      the raw content to analyze.
 |  
 |      If 'file', the sequence items must have a 'read' method (file-like
 |      object) that is called to fet

In [49]:
pipeline = Pipeline([
    ('features', FeatureUnion([
        ('tf_idf', Pipeline([
            ('counts', CountVectorizer()),
            ('tf_idf', TfidfTransformer()),
            ('chi', SelectKBest(chi2, k=20000))
        ])),
        ('len_stats', StatsFeatures())
    ])),
    ('classifier', XGBClassifier(max_depth=7,objective='multi:softmax', num_class=2))
])

pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('tf_idf', Pipeline(memory=None,
     steps=[('counts', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0...tate=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1))])

In [51]:
print(pipeline.score(X_train, y_train))

0.9536668142245831


## test

In [52]:
y_pred_class = pipeline.predict(X_test)
print('accuracy_score: ', metrics.accuracy_score(y_test, y_pred_class)) # 指所有分类正确的百分比
print(metrics.classification_report(y_test, y_pred_class))
print('confusion_matrix: ')
print( metrics.confusion_matrix(y_test, y_pred_class))

accuracy_score:  0.8911022576361222
             precision    recall  f1-score   support

          0       0.88      0.94      0.91       862
          1       0.92      0.82      0.87       644

avg / total       0.89      0.89      0.89      1506

confusion_matrix: 
[[813  49]
 [115 529]]


In [54]:
# 上一版模型 
y_pred_class = pipeline_old.predict(X_test)
print('accuracy_score: ', metrics.accuracy_score(y_test, y_pred_class)) # 指所有分类正确的百分比
print(metrics.classification_report(y_test, y_pred_class))
print('confusion_matrix: ')
print( metrics.confusion_matrix(y_test, y_pred_class))

accuracy_score:  0.8652058432934927
             precision    recall  f1-score   support

          0       0.88      0.89      0.88       862
          1       0.85      0.84      0.84       644

avg / total       0.87      0.87      0.87      1506

confusion_matrix: 
[[764  98]
 [105 539]]


# 模型保存

In [55]:
from sklearn.externals import joblib
joblib.dump(pipeline, "model/circ_cor_0703.pkl.z")  

['model/circ_cor_0703.pkl.z']

# 保存本文件

In [56]:
if 0:
    import datetime as dt
    
    def output_HTML(read_file, output_file):
        from nbconvert import HTMLExporter
        import codecs
        import nbformat
        exporter = HTMLExporter()
        # read_file is '.ipynb', output_file is '.html'
        output_notebook = nbformat.read(read_file, as_version=4)
        output, resources = exporter.from_notebook_node(output_notebook)
        codecs.open(output_file, 'w', encoding='utf-8').write(output)

    html_file_folder = 'html_files'
    if not os.path.exists(html_file_folder):
        os.makedirs(html_file_folder)

    today = dt.datetime.now().strftime('%Y%m%d')
    current_file = 'circ_cor_model_2_train.ipynb'
    output_file = 'html_files\%s_%s.html'%(os.path.splitext(current_file)[0], today)
    output_HTML(current_file, output_file)